In [2]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm

In [3]:
load_dotenv()

True

In [4]:
data = pd.read_excel(r"C:\Users\smrit\Work\Kenpath\NOS-QUALS\Sales NOS\parsed_nos_data.xlsx")

In [5]:
data.columns

Index(['uuid', 'nos_id', 'industry', 'title', 'type', 'text'], dtype='object')

## Create Embeddings

In [5]:
#text-embedding-ada-002
#text-embedding-3-small


In [6]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  
)

def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

data['embeddings'] = data['text'].apply(lambda x: get_embedding(x))

## Create Vectors list

In [7]:
vectors = [
    {
        'id': str(row['uuid']),  
        'values': row['embeddings'], 
        'metadata': {
            'nos_id': row['nos_id'],
            'industry':row['industry'],
            'title': row['title'],
            'type': row['type'],
            'text': row['text']
        }
    }
    for _, row in data.iterrows()
]

In [8]:
len(vectors)

81

In [9]:
os.chdir(r'C:\Users\smrit\Work\Kenpath\zavmo-api')

## Creating index and upsert vectors

In [15]:
from pinecone_index import PineconeIndex

# Initialize PineconeIndex
pinecone_index = PineconeIndex(index_name='test-nos', dimension=1536)

INFO:pinecone_index:Creating index: test-nos


In [12]:
#pinecone_index.delete_all()

In [ ]:
#delete index
pinecone_index.delete_index()

In [19]:
# Upsert vectors into the index
pinecone_index.upsert_vectors(vectors)

INFO:pinecone_index:Upserting 81 vectors into the index


In [20]:
# Get the number of vectors in the index
vector_count = pinecone_index.get_vector_count()
print(f"Number of vectors in the index: {vector_count}")

INFO:pinecone_index:Total vectors in the index: 81


Number of vectors in the index: 81


### Querying 

In [ ]:
# Example query
query_vector = data['embeddings'][0]  
search_results = pinecone_index.search_items([query_vector])

# Print search results
for result in search_results:
    print(f"Found match: {result['id']} with score: {result['score']}, metadata: {result['metadata']}")

In [21]:
question = "I am a sales manager, what are the required skills I should know and learn according to NOS?"
query_vector = get_embedding(question)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [22]:
search_results = pinecone_index.search_items([query_vector])

for result in search_results:
    print(f"Found match: {result['id']} with score: {result['score']}, metadata: {result['metadata']}")

INFO:pinecone_index:Searching the index with 1 query vectors


Found match: 42a96032-f8a2-4ed9-8e2b-af3ace5b5b5f with score: 0.610552728, metadata: {'industry': 'Sales', 'nos_id': 'INSSAL014', 'text': 'Performance criteria\nYou must be able to:\n1.    use sales-related information for planning and implementing sales activities \n2.    store information on the sales technology systems, checking its quality, validity \nand reliability \n3.    maintain the sales-related information stored in the technology systems, ensuring \nit is accurate and up to date \n4.    use tools and technology systems for automation of sales-related tasks and \nactivities \n5.    share sales-related information for collaboration with members of sales team \n6.    use lead generation automation technology to qualify the leads \n7.    collate potential leads and prioritise these in accordance with sales targets \n8.    summarise information appropriately from sales systems for reporting to \nmanagement \n9.    follow up incoming leads and assign the ownership of quality lead

In [33]:
#os.chdir(r"C:\Users\smrit\Work\Kenpath\zavmo-api\zavmo\helpers")

### Retrieving nos documents passing currentrole and filter=Sales

In [31]:
import search
results = search.fetch_nos_text(industry="Sales", current_role="Sales Manager")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [32]:
results

['Performance criteria\nYou must be able to:\n1.    identify the target markets for sales and prepare for sales activities \n2.    identify customers to contact and the range of products and services \n3.    prepare and follow call plans, email campaigns and other means of communication \nwith customers \n4.    help the customer become comfortable with making an investment into your \nproducts of services, understanding the value of doing so \n5.    prepare sales materials and accompanying messages during contacts with \ncustomers \n6.    adhere to your organisation’s dress code while selling to customers \n7.    agree procedures for collecting contact details of potential customers \n8.    adhere to health, safety and security requirements appropriate to the face-to-face \nsales environment \n9.    contact customers through relevant means of communication \n10.  identify customers’ requirements through questioning and confirm these by \nsummarising their needs and interests \n11.  emp